# Creating, Distributing, and Testing Julia Packages with Binary Dependencies

JuliaCon 2016 Workshop

Tony Kelman (`@tkelman`)

- So you wrote or want to use a Julia package that [calls into C or Fortran](http://docs.julialang.org/en/release-0.4/manual/calling-c-and-fortran-code/).
- How do you go from "`ccall` works on my machine" to making your package easy to install for anyone?
- You want `Pkg.add("MyPackage")` to find, build, or download a copy of the library that will work on:
  - Linux (many distributions, Ubuntu, CentOS, etc) on `x86_64` architecture
  - Mac OS X
  - Windows (7,8,10, with 64 bit and 32 bit Julia)
  - More exotic OS or architecture combinations (won't cover in much detail)
- Ideally without requiring the user to have root/admin permissions

In [2]:
include("clone_example.jl")

Already up-to-date.
Already up-to-date.
